In [1]:
import pymongo
import charts

Server running in the folder /home/isaac at 127.0.0.1:60851


In [2]:
client = pymongo.MongoClient('localhost', 27017)
ganji = client['ganji']
item_sample = ganji['item_sample']

In [27]:
for i in item_sample.find({},{'look':1}).limit(300):
    print(i)

{'_id': ObjectId('5698f524a98063dbe9e91ca8'), 'look': '-'}
{'_id': ObjectId('5698f525a98063dbe4e91ca8'), 'look': '-'}
{'_id': ObjectId('5698f525a98063dbe6e91ca8'), 'look': '-'}
{'_id': ObjectId('5698f525a98063dbe7e91ca8'), 'look': '-'}
{'_id': ObjectId('5698f525a98063dbe8e91ca8'), 'look': '-'}
{'_id': ObjectId('5698f525a98063dbe9e91ca9'), 'look': '-'}
{'_id': ObjectId('5698f525a98063dbe6e91ca9'), 'look': '-'}
{'_id': ObjectId('5698f525a98063dbe8e91ca9'), 'look': '-'}
{'_id': ObjectId('5698f525a98063dbe7e91ca9'), 'look': '9成新'}
{'_id': ObjectId('5698f525a98063dbe5e91ca8'), 'look': '-'}
{'_id': ObjectId('5698f526a98063dbe9e91caa'), 'look': '-'}
{'_id': ObjectId('5698f526a98063dbe7e91caa'), 'look': '-'}
{'_id': ObjectId('5698f526a98063dbe8e91caa'), 'look': '-'}
{'_id': ObjectId('5698f526a98063dbe6e91caa'), 'look': '95成新'}
{'_id': ObjectId('5698f527a98063dbe9e91cab'), 'look': '-'}
{'_id': ObjectId('5698f527a98063dbe4e91ca9'), 'look': '9成新'}
{'_id': ObjectId('5698f527a98063dbe6e91cab'), 'lo

In [8]:
pipeline1 = [
    #$all:包含
    {'$match': {'$and':[{'pub_date':{'$gte':'2015.12.25', '$lte': '2015.12.27'}},{'area':{'$all':['朝阳']}}]}},#用比较运算符选择日期
    {'$group': {'_id': {'$slice':['$cates',2,1]}, 'counts':{'$sum':1}}},
    {'$sort': {'counts':-1}},
    {'$limit':3}
]

for i in item_sample.aggregate(pipeline1):
    print(i)

{'_id': ['北京二手家电'], 'counts': 175}
{'_id': ['北京二手服装/鞋帽/箱包'], 'counts': 105}
{'_id': ['北京二手办公用品/设备'], 'counts': 86}


In [11]:
def data_gen(date1, date2, area, limit):
    pipeline1 = [
    #$all:包含
    {'$match': {'$and':[{'pub_date':{'$gte':date1, '$lte': date2}},{'area':{'$all':area}}]}},#用比较运算符选择日期
    {'$group': {'_id': {'$slice':['$cates',2,1]}, 'counts':{'$sum':1}}},
    {'$sort': {'counts':-1}},
    {'$limit':limit}
]

    for i in item_sample.aggregate(pipeline1):
        data = {
            'name': i['_id'][0],
            'data': [i['counts']],
            'type': 'column'
        }
        yield data

In [12]:
for i in data_gen('2015.12.25', '2015.12.27', ['朝阳'], 3):
    print(i)

{'name': '北京二手家电', 'data': [175], 'type': 'column'}
{'name': '北京二手服装/鞋帽/箱包', 'data': [105], 'type': 'column'}
{'name': '北京二手办公用品/设备', 'data': [86], 'type': 'column'}


In [17]:
series = [i for i in data_gen('2015.12.25', '2015.12.27', ['朝阳'], 10)]
options = {
    'chart'   : {'zoomType':'xy'},
    'title'   : {'text': '发帖量最大的类目'},
    'subtitle': {'text': '可视化统计图表'},
    'yAxis'   : {'title': {'text': '数量'}}
    }
charts.plot(series, options = options, show = 'inline')

In [47]:
#某时段内各个类目对应成色的平均价格
#nin: not in
pipeline2 = [
    {'$match': {'$and':[{'pub_date':{'$gte':'2015.12.25', '$lte': '2015.12.27'}},
                        {'cates':{'$all':['北京二手手机']}},
                        {'look': {'$nin' :['-']}}
                      ]}},
    {'$group': {'_id': '$look', 'avg_price':{'$avg':'$price'}}},
    {'$sort' :{'avg_price':-1}}
]



In [48]:
for i in item_sample.aggregate(pipeline2):
    print(i)

{'_id': '全新', 'avg_price': 1849.7894736842106}
{'_id': '99成新', 'avg_price': 1704.5}
{'_id': '95成新', 'avg_price': 1626.923076923077}
{'_id': '9成新', 'avg_price': 1187.5151515151515}
{'_id': '8成新', 'avg_price': 730.0}
{'_id': '7成新及以下', 'avg_price': 300.0}
{'_id': '报废机/尸体', 'avg_price': 220.0}


In [51]:
def data_gen2(date1, date2, cates):
    pipeline2 = [
    {'$match': {'$and':[{'pub_date':{'$gte':date1, '$lte': date2}},
                        {'cates':{'$all':cates}},
                        {'look': {'$nin' :['-']}}
                      ]}},
    {'$group': {'_id': '$look', 'avg_price':{'$avg':'$price'}}},
    {'$sort' :{'avg_price':1}}
]
    for i in item_sample.aggregate(pipeline2):
        yield i['avg_price']

In [52]:
data = [i for i in data_gen2('2015.12.24','2016.01.10',['北京二手手机'])]
options = {
    'title': {'text': '新旧-价格'},
    'xAxis'   : {'categories': ['报废机/尸体','7成新及以下','8成新','9成新','95成新','99成新', '全新']},
    'yAxis'   : {'title': {'text': '价格'}},
    
}

charts.plot(data,show='inline', options=options)

In [ ]:
#如何导出CSV
#mongoexport -d database -c collection -o output/path.csv